In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import zipfile
import re
import json
import dateparser
import collections
from lxml import etree
from xml.etree.ElementTree import XML
from random import shuffle
docxFileName = "quick_section4.docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

In [11]:
mainstation_xpath = './w:r[not(preceding-sibling::w:r//w:t)][not(w:rPr/w:sz[@w:val="16"])][w:rPr/w:b[not(@w:val="0")]]/w:t[1]'
mainstation_xpath2 = './w:r[not(preceding-sibling::w:r//w:t)][not(w:rPr/w:sz[@w:val="16"])][../w:pPr[w:pStyle]/w:rPr/w:b[@w:val="0"]]/w:t[1]'

first_token_para_xpath = './w:r[//w:t]/w:t[1]'

stations = pd.DataFrame(columns=['station','type','description'])

def is_upper(para):
    if para.xpath(first_token_para_xpath, namespaces=ns):
        if para.xpath(first_token_para_xpath, namespaces=ns)[0].text.isupper():
            return 'station'
        else: 
            return 'new_line'

mainstation = ""
initial_letter = ""
lowerstation = ""
dText = dict()
counter = 0
for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    text = para.xpath('./w:r/w:t', namespaces=ns)
    description = " ".join([t.text for t in text])
    if is_upper(para) == 'station':
        mainxpath = para.xpath(mainstation_xpath2, namespaces=ns)
        # Filter out wrongly formatted substations:
        if mainxpath and len(mainxpath[0].text) > 1 and not mainxpath[0].text.startswith(initial_letter + " "):
            print(mainxpath[0].text)
            counter += 1
            mainstation = mainxpath[0].text
            initial_letter = mainstation[0]
    description = description.lstrip('\x01').strip()
    if description:
        if (counter, mainstation) in dText:
            dText[(counter, mainstation)].append(description)# += "\n" + description
        else:
            description = re.sub('^(' + re.escape(mainstation) + ')', '\1', description).lstrip('\x01').strip()
            description = re.sub(r" +", " ", description).lstrip('\x01').strip()
            if description:
                dText[(counter, mainstation)] = [description]

ABERBEEG
ADAM STREET 
AINSWORTH ROAD 
ALDWYCH
ALEXANDRA PARADE
ALFRETON
ANNITSFORD
ANSTRUTHER
ASHURST
AYOT
BACHAN
BACHE
BACK OF LAW 
BAGILLT
BARNES
BARRS COURT JUNCTION
BEDLINGTON
BETCHWORTH
BETLEY ROAD
BIDFORD-ON-AVON
BIGLIS/B JUNCTION
BODDAM
BO’NESS
BONNINGTON
BOTTOM OF SUTTON INCLINE
BOWES BRIDGE
BRAMSHOT
BRIDLINGTON
BRITISH STEEL
BRYNDERWEN
BRYNGLAS
BURNT FEN
BUSHEY PARK 
CAERLEON
CAMBRIDGE HEATH
CAMPBELTOWN
CAMPERDOWN
CANLEY
CANNON STREET ROAD
CANTLEY
CAPEL BANGOR
CARDIFF
CARDIGAN
CARLISLE
CARLTON MAIN COLLIERY
CARLTON/CARLTON IRONWORKS
CARLUKE
CARMARTHEN
CARNO
CARSHALTON
CARSTAIRS
CASTLE DONINGTON & SHARDLOW
CASTLE DOUGLAS
CASTLE EDEN
CASTLEMILK
CASTLETHORPE
CATHCART
CEFN COED
CELYNEN
CENTRAL
CHALK FARM
CHALKWELL
CHAPELTON
CHAPELTOWN
CHARFIELD
CHARING CROSS
CHATHAM
CHATHILL
CHEADLE
CHELLASTON & SWARKESTONE
CHEQUERBENT
CHERRY BURTON
CHESSINGTON
CHESTER ROAD
CHESTER-LE-STREET
CHISELDON
CHISLEHURST
CHISLET 
CHITTENING
CHORLEYWOOD
CHURCH FENTON 
CHURCH MANOR WAY
CHURCH STRETTON
CHURC

In [3]:
dStations = collections.OrderedDict(dText)

In [4]:
indices = []
stations = []
descriptions = []
for k in dStations:
    indices.append(k[0])
    stations.append(k[1])
    descriptions.append(dStations[k])

In [5]:
stationdf = pd.DataFrame(columns=["Index", "Station", "Description"])
stationdf["Index"] = indices
stationdf["Station"] = stations
stationdf["Description"] = descriptions
stationdf = stationdf.set_index("Index")

In [6]:
stationdf.to_pickle('quick_processed.pkl')

In [7]:
stationdf_sample = stationdf.sample(n=100, random_state=42)

# Get 100 random entries:
for i, row in stationdf_sample.iterrows():
    with open("to_annotate/" + str(i) + ".txt", "w") as fw:
        fw.write("[[[STATION: " + row["Station"] + "]]]")
        for d in row["Description"]:
            fw.write("\n" + d)

# Get 25 long entries:
stationdf_sample = stationdf.sample(frac=1, random_state=42)
counter = 1
for i, row in stationdf_sample.iterrows():
    if len(row['Description']) >= 10:
        with open("to_annotate/" + str(i) + ".txt", "w") as fw:
            fw.write("[[[STATION: " + row["Station"] + "]]]")
            for d in row["Description"]:
                fw.write("\n" + d)
        if counter >= 25:
            break
        counter += 1

FileNotFoundError: [Errno 2] No such file or directory: 'to_annotate/173.txt'

In [47]:
## testing
unpickled_df = pd.read_pickle("quick_processed.pkl")

In [48]:
## testing
unpickled_df.loc[unpickled_df.Station.str.startswith('ABERAVON', na=False)]

,Station,Description
Index,,
25,ABERAVON,"[[RSB] {map 85}., A SEA SIDE op 1 March 1899..."


In [73]:
## testing
h1 = et.xpath('//./w:r[preceding-sibling::w:pPr/w:pStyle[@w:val="Heading1"]][1]/w:t[1]', namespaces=ns)
h2 = et.xpath('//./w:r[preceding-sibling::w:pPr/w:pStyle[@w:val="Heading2"]][1]/w:t[1]', namespaces=ns)
h3 = et.xpath('//./w:r[preceding-sibling::w:pPr/w:pStyle[@w:val="Heading3"]][1]/w:t[1]', namespaces=ns)
h4 = et.xpath('//./w:r[preceding-sibling::w:pPr/w:pStyle[@w:val="Heading4"]][1]/w:t[1]', namespaces=ns)
main1 = et.xpath('//.'+mainstation_xpath, namespaces=ns)
main2 = et.xpath('//.'+mainstation_xpath2, namespaces=ns)

mains = list(unpickled_df.Station)
print('Heading1:\n')
for a in h1:
    if a not in mains:
        print(a.text)
print()
print('Heading2:\n')
for a in h2:
    if a not in mains:
        print(a.text)
print()
print('Heading3:\n')
for a in h3:
    if a not in mains:
        print(a.text)
print()
print('Heading4:\n')
for a in h4:
    if a not in mains:
        print(a.text)

In [ ]:
#testing
main1 = et.xpath(mainstation_xpath, namespaces=ns)
main2 = et.xpath(mainstation_xpath2, namespaces=ns)

main1